In [2]:
from Bio import SeqIO
import os,sys
import pandas as pd
import phylopandas as ph

In [103]:

def getgene(fasta_file, anno_list):
    f = open(fasta_file, "w")
    for rec in SeqIO.parse("BetaCoV_0529.gb", "gb"):
        for feature in rec.features:
            for key, val in feature.qualifiers.items():
                if feature.type == "CDS":
                    if any (s in val for s in anno_list):
                        print (">" + rec.id, file=f )
                        print (feature.location.extract(rec).seq,file=f)
    f.close()

In [104]:
def duplicate_remover(fasta_file):
    df = ph.read_fasta(fasta_file)
    df = df.filter(['id','sequence'], axis=1)
    df = df.drop_duplicates()
    df.to_csv("temp.tab", sep="\t",index = False,header=False)
    df2= SeqIO.parse("temp.tab", "tab")
    SeqIO.write(df2, "rm"+fasta_file, "fasta")
    os.remove("temp.tab")
    print("duplicate records removed!\n"+ str(len(df.index))+" unique records saved from " + fasta_file)

In [5]:
## process annotation table 

ano = pd.read_csv('BetaCoV_0529 Annotations.csv')
CDS = ano.loc[ano['Type'] == 'CDS']
CDS_name = CDS.groupby(['Name']).count()
CDS_name

/Users/jianichen1/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Sequence Name  Type  Minimum  Maximum  \
Name                                                                           
1 CDS                                             13    13       13       13   
10.1 kDa nonstructural protein CDS                 1     1        1        1   
12.6 kD nonstructural protein CDS                  2     2        2        2   
12.7 kD non-structural protein CDS                 5     5        5        5   
12.7 kDa accessory protein CDS                     1     1        1        1   
...                                              ...   ...      ...      ...   
unnamed protein product; orf3a CDS                 1     1        1        1   
unnamed protein product; orf3b CDS                 1     1        1        1   
unnamed protein product; orf6 CDS                  1     1        1        1   
unnamed protein product; rat XT-2 CDS              1     1        1        1   
unnamed protein product; rat XT-I CDS              1     1        1        1   

                                       Length  # Intervals  Direction  \
Name                                                                    
1 CDS                                      13           13         13   
10.1 kDa nonstructural protein CDS          1            1          1   
12.6 kD nonstructural protein CDS           2            2          2   
12.7 kD non-structural protein CDS          5            5          5   
12.7 kDa accessory protein CDS              1            1          1   
...                                       ...          ...        ...   
unnamed protein product; orf3a CDS          1            1          1   
unnamed protein product; orf3b CDS          1            1          1   
unnamed protein product; orf6 CDS           1            1          1   
unnamed protein product; rat XT-2 CDS       1            1          1   
unnamed protein product; rat XT-I CDS       1            1          1   

                                       Sequence (with extension)  \
Name                                                               
1 CDS                                                          0   
10.1 kDa nonstructural protein CDS                             0   
12.6 kD nonstructural protein CDS                              0   
12.7 kD non-structural protein CDS                             0   
12.7 kDa accessory protein CDS                                 0   
...                                                          ...   
unnamed protein product; orf3a CDS                             0   
unnamed protein product; orf3b CDS                             0   
unnamed protein product; orf6 CDS                              0   
unnamed protein product; rat XT-2 CDS                          0   
unnamed protein product; rat XT-I CDS                          0   

                                       Length (with extension)  
Name                                                            
1 CDS                                                        0  
10.1 kDa nonstructural protein CDS                           0  
12.6 kD nonstructural protein CDS                            0  
12.7 kD non-structural protein CDS                           0  
12.7 kDa accessory protein CDS                               0  
...                                                        ...  
unnamed protein product; orf3a CDS                           0  
unnamed protein product; orf3b CDS                           0  
unnamed protein product; orf6 CDS                            0  
unnamed protein product; rat XT-2 CDS                        0  
unnamed protein product; rat XT-I CDS                        0  

[629 rows x 9 columns]

In [6]:

CDS_name.to_csv("CDSname_0529.csv")

In [105]:
S={'S',
'spike protein',
'S protein',
'spike glycoprotein',
'spike glycoprotein precursor',
'spike',
'membrane glycoprotein',
'spike glycoprotein S',
'putative spike glycoprotein',
'Spike protein',
'putative E2 glycoprotein precursor',
'spike surface glycoprotein',
'S glycoprotein',
'S peplomer polypeptide precursor',
'membrane associated glycoprotein E2 precursor',
'putative spike glycoprotein S',
'spike glycoprotein (S)',
'spike structural protein',
'surface glycoprotein S',
'surface projection glycoprotein' }

In [106]:
getgene("S_0529.fasta",S)

In [107]:

duplicate_remover("S_0529.fasta")

duplicate records removed!
11799 unique records saved from S_0529.fasta


In [108]:

## duplicate S need to be further removed, the shorter one will be removed

df = ph.read_fasta("rmS_0529.fasta")
df['seq_length'] = df['sequence'].str.len()
##sort by length
df = df.sort_values('seq_length')
## keep the last one with the identical id
df = df.drop_duplicates(subset='id',keep = "last") 
df = df.filter(['id','sequence'], axis=1)
print(len(df.index))
df.to_csv("temp.tab", sep="\t",index = False,header=False)
df2= SeqIO.parse("temp.tab", "tab")
SeqIO.write(df2, "rm2S_0529.fasta", "fasta")
os.remove("temp.tab")

7518


In [109]:

## filter by length and propotion of "N"
## the minimal length of sequence

##from https://biopython.org/wiki/Sequence_Cleaner
def sequence_cleaner(fasta_file, min_length=0, por_n=100):
    output_file = open("clear_" + fasta_file, "w")

    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        if (len(seq_record.seq) >= min_length and (float((seq_record.seq).count("N"))/float(len(seq_record.seq)))*100 <= por_n):
            output_file.write(">" + seq_record.id + "\n" + str(seq_record.seq) + "\n")

    print("CLEAN!!!\nPlease check clear_" + fasta_file)

In [110]:
sequence_cleaner("rm2S_0529.fasta",2000,50)

CLEAN!!!
Please check clear_rm2S_0529.fasta


In [111]:
## get metadata table for S gene
meta = pd.read_csv("BetaCoV_0529.csv")
    
df = ph.read_fasta("clear_rm2S_0529.fasta")
df = df.filter(['id','sequence'], axis=1)
df[['Accession','version']] = df.id.str.split(".",expand=True)
fasta=pd.merge(df, meta, how='left', left_on='Accession', right_on='Accession')
fasta=fasta.drop(columns=['sequence'])
fasta.to_csv("BetaCoV_S_0529.csv",index = False)

In [112]:
def irma_species(excel_file, fasta_file):
    seq = ph.read_fasta("clear_rm2S_0529.fasta")
    seq = seq.filter(['id','sequence'], axis=1)
    df = pd.read_excel(excel_file, header = None)
    df.rename(columns ={0: 'id'}, inplace =True)

    fasta =pd.merge(df, seq, how='left', left_on='id', right_on='id')
    fasta = fasta.filter(['id','sequence'], axis=1)
    fasta=fasta.dropna(subset=['sequence'])

    num=fasta
    fasta.to_csv("temp.tab", sep="\t",index = False,header=False)
    df2= SeqIO.parse("temp.tab", "tab")
    SeqIO.write(df2, fasta_file, "fasta")
    os.remove("temp.tab")
    print(str(len(num.index)) + " in " + fasta_file)

In [113]:
irma_species("HKU1.xlsx","HKU1.fasta")

58 in HKU1.fasta


In [114]:
irma_species("HKU4.xlsx","HKU4.fasta")

24 in HKU4.fasta


In [115]:
irma_species("HKU5.xlsx","HKU5.fasta")

25 in HKU5.fasta


In [136]:
irma_species("HKU9.xlsx","HKU9.fasta")

10 in HKU9.fasta


In [133]:
irma_species("BetaCoV1.xlsx","BetaCoV1.fasta")

309 in BetaCoV1.fasta


In [118]:
irma_species("CoV19.xlsx","CoV19.fasta")

4259 in CoV19.fasta


In [119]:
irma_species("OC43.xlsx","OC43.fasta")

538 in OC43.fasta


In [120]:
irma_species("murine.xlsx","murine.fasta")

50 in murine.fasta


In [121]:
irma_species("bat_HP_zhejiang.xlsx","bat_HP_zhejiang.fasta")

2 in bat_HP_zhejiang.fasta


In [122]:
irma_species("pangolin.xlsx","pangolin.fasta")

6 in pangolin.fasta


In [123]:
irma_species("MERS.xlsx","MERS.fasta")

806 in MERS.fasta


In [124]:
irma_species("SARS.xlsx","SARS.fasta")

282 in SARS.fasta


In [126]:
irma_species("BetaCoV_sp.xlsx","BetaCoV_sp.fasta")

11 in BetaCoV_sp.fasta


In [127]:
irma_species("HKU14.xlsx","HKU14.fasta")

5 in HKU14.fasta


In [129]:
irma_species("rat.xlsx","rat.fasta")

5 in rat.fasta


In [130]:
irma_species("longquan_murine.xlsx","longquan_murine.fasta")

4 in longquan_murine.fasta


In [131]:
irma_species("hedgedog.xlsx","hedgedog.fasta")

6 in hedgedog.fasta


In [146]:
irma_species("procine.xlsx","procine.fasta")

21 in procine.fasta


In [135]:
irma_species("equine.xlsx","equine.fasta")

8 in equine.fasta


In [137]:
irma_species("HKU14.xlsx","HKU14.fasta")

5 in HKU14.fasta


In [138]:
irma_species("HKU23.xlsx","HKU23.fasta")

15 in HKU23.fasta


In [139]:
irma_species("HKU24.xlsx","HKU24.fasta")

4 in HKU24.fasta


In [140]:
irma_species("HKU25.xlsx","HKU25.fasta")

2 in HKU25.fasta


In [141]:
irma_species("SARS_like.xlsx","SARS_like.fasta")

60 in SARS_like.fasta


In [144]:
irma_species("batCoV.xlsx","batCoV.fasta")

5 in batCoV.fasta


In [147]:
irma_species("Rou_bat.xlsx","Rou_bat.fasta")

3 in Rou_bat.fasta


In [148]:
irma_species("HKU15.xlsx","HKU15.fasta")

3 in HKU15.fasta


In [149]:
irma_species("HKU16.xlsx","HKU16.fasta")

2 in HKU16.fasta


In [150]:
irma_species("HKU17.xlsx","HKU17.fasta")

2 in HKU17.fasta


In [151]:
irma_species("HKU18.xlsx","HKU18.fasta")

2 in HKU18.fasta


In [152]:
irma_species("HKU19.xlsx","HKU19.fasta")

2 in HKU19.fasta


In [153]:
irma_species("HKU20.xlsx","HKU20.fasta")

2 in HKU20.fasta


In [154]:
irma_species("HKU21.xlsx","HKU21.fasta")

2 in HKU21.fasta


In [155]:
irma_species("kenya_bat.xlsx","kenya_bat.fasta")

2 in kenya_bat.fasta


In [157]:
irma_species("pipist.xlsx","pipist.fasta")

0 in pipist.fasta
